This notebook will showcase the development and implementation of a deep learning convolutional neural network to
solve a the facial expression classification problem.

# Downloading & Exploring Dataset

This model will use a dataset of facial expression images that are for training and validation. There are five subdirectories, each one being linked to a class.

Here is a breakdown of the facial expressions directory and its subdirectories:

```
facial_expressions/
  train/
    suprise/
    anger/
    happiness/
    sadness/
    fear/
    disgust/
  validation/
    suprise/
    anger/
    happiness/
    sadness/
    fear/
    disgust/
```

Download the data:

In [1]:
import tensorflow as tf
import pathlib

dataset_url = "https://storage.googleapis.com/cp468-group-1/facial_expressions_v2.zip"
data_dir = tf.keras.utils.get_file(origin=dataset_url, extract=True, archive_format='zip')
data_dir = pathlib.Path(data_dir).with_suffix('')


43653920/43653920 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Explore the data:

In [2]:
import os
import pathlib

extracted_dir = pathlib.Path(data_dir).parent / 'facial_expressions'

training_dir = extracted_dir / 'train'
training_surprise_dir = training_dir / 'surprise'
training_anger_dir = training_dir / 'anger'
training_happiness_dir = training_dir / 'happiness'
training_fear_dir = training_dir / 'fear'
training_disgust_dir = training_dir / 'disgust'
training_neutral_dir = training_dir / 'neutral'

validation_dir = extracted_dir / 'validation'
validation_surprise_dir = validation_dir / 'surprise'
validation_anger_dir = validation_dir / 'anger'
validation_happiness_dir = validation_dir / 'happiness'
validation_fear_dir = validation_dir / 'fear'
validation_disgust_dir = validation_dir / 'disgust'
validation_neutral_dir = validation_dir / 'neutral'

# Count images function
def count_images(directory):
  image_count = len(list(directory.glob('*.jpg')))
  return image_count

# print(f"Training Dataset Image Count: {count_images(training_dir)}")
print(f"Training / Surprise: {count_images(training_surprise_dir)}")
print(f"Training / Anger: {count_images(training_anger_dir)}")
print(f"Training / Happiness: {count_images(training_happiness_dir)}") # - 6k
print(f"Training / Fear: {count_images(training_fear_dir)}")
print(f"Training / Disgust: {count_images(training_disgust_dir)}")
print(f"Training / Neutral: {count_images(training_neutral_dir)}")

# print(f"Validation Dataset Image Count: {count_images(validation_dir)}")
print(f"validation / Surprise: {count_images(validation_surprise_dir)}")
print(f"validation / Anger: {count_images(validation_anger_dir)}")
print(f"validation / Happiness: {count_images(validation_happiness_dir)}")
print(f"validation / Fear: {count_images(validation_fear_dir)}")
print(f"validation / Disgust: {count_images(validation_disgust_dir)}")
print(f"validation / Neutral: {count_images(validation_neutral_dir)}")

Training / Surprise: 831
Training / Anger: 1225
Training / Happiness: 847
Training / Fear: 1657
Training / Disgust: 436
Training / Neutral: 2112
validation / Surprise: 1246
validation / Anger: 958
validation / Happiness: 1774
validation / Fear: 1024
validation / Disgust: 111
validation / Neutral: 1233


# Normalizing Data  

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale for validation
validation_datagen = ImageDataGenerator(rescale=1./255)

# Use the extracted_dir variable which points to the correct directory
train_generator = train_datagen.flow_from_directory(
    os.path.join(extracted_dir, 'train'), # Change data_dir to extracted_dir
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(extracted_dir, 'validation'), # Change data_dir to extracted_dir
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)

Found 11938 images belonging to 7 classes.
Found 7593 images belonging to 7 classes.


# Setting up the pre-trained model (Inception V3)

In [11]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models

# Load the pre-trained VGG16 model
base_model = InceptionV3(input_shape=(299, 299, 3), include_top=False, weights='imagenet')

# Freeze the pre-trained layers
base_model.trainable = False

Creating the training datasets:

In [12]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    training_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(299, 299),
    batch_size=64
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    training_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(299, 299),
    batch_size=64
)

class_names = train_ds.class_names
print(class_names)

# Check the number of classes in your dataset
num_classes = len(class_names)
print("Number of classes:", num_classes)

Found 11938 files belonging to 7 classes.
Using 9551 files for training.
Found 11938 files belonging to 7 classes.
Using 2387 files for validation.
['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
Number of classes: 7


Adding custom layers to the model:

In [13]:
# Add custom layers on top of the base model
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

# Create the final model, and modify the final layer of your model to match the number of classes
model = models.Model(inputs=base_model.input, outputs=layers.Dense(num_classes, activation='softmax')(x))

Compile the model:

In [14]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train the model:

In [ ]:
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=validation_generator
)

Epoch 1/3
